# AdaBoost Classifier From Scratch

AdaBoost (Adaptive Boosting) is an ensemble method that combines multiple "weak learners" (usually Decision Stumps) into one strong learner by focusing on previously misclassified samples.

## Key Concepts:
- **Weak Learner**: A model that performs slightly better than random guessing
- **Decision Stump**: A Decision Tree with only one split (depth=1)
- **Sample Weights**: Misclassified samples get higher weights in the next round
- **Alpha (Learner Weight)**: How much we trust each weak learner

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split
from sklearn.ensemble import AdaBoostClassifier as SklearnAda

## 1. Decision Stump Implementation

A decision stump is a single-split tree. To train it with weights, we search for the split that minimizes the **weighted error**.

In [ ]:
class DecisionStump:
    def __init__(self):
        self.feature_idx = None
        self.threshold = None
        self.polarity = 1 # Direction of < threshold
        self.alpha = None

    def predict(self, X):
        n_samples = X.shape[0]
        X_column = X[:, self.feature_idx]
        predictions = np.ones(n_samples)
        if self.polarity == 1:
            predictions[X_column < self.threshold] = -1
        else:
            predictions[X_column >= self.threshold] = -1
        return predictions

## 2. AdaBoost Implementation

In [ ]:
class AdaBoostClassifier:
    def __init__(self, n_estimators=50):
        self.n_estimators = n_estimators
        self.stumps = []

    def fit(self, X, y):
        n_samples, n_features = X.shape
        # Weights initialized to 1/N
        w = np.full(n_samples, (1 / n_samples))
        
        self.stumps = []

        for _ in range(self.n_estimators):
            stump = DecisionStump()
            min_error = float('inf')

            # Find best split for weighted error
            for f_idx in range(n_features):
                X_column = X[:, f_idx]
                thresholds = np.unique(X_column)
                for threshold in thresholds:
                    for polarity in [1, -1]:
                        # Predict
                        predictions = np.ones(n_samples)
                        if polarity == 1:
                            predictions[X_column < threshold] = -1
                        else:
                            predictions[X_column >= threshold] = -1

                        # Weighted error calculation
                        error = sum(w[y != predictions])
                        
                        if error < min_error:
                            min_error = error
                            stump.threshold = threshold
                            stump.feature_idx = f_idx
                            stump.polarity = polarity

            # Calculate Alpha (Amount of Say)
            # EPS to avoid division by zero
            EPS = 1e-10
            stump.alpha = 0.5 * np.log((1.0 - min_error + EPS) / (min_error + EPS))

            # Update Weights
            predictions = stump.predict(X)
            w *= np.exp(-stump.alpha * y * predictions)
            # Normalize
            w /= np.sum(w)

            self.stumps.append(stump)

    def predict(self, X):
        stump_preds = [stump.alpha * stump.predict(X) for stump in self.stumps]
        y_pred = np.sum(stump_preds, axis=0)
        return np.sign(y_pred)

    def score(self, X, y):
        return np.mean(self.predict(X) == y)

## 3. Testing on Moons Dataset

In [ ]:
X, y = make_moons(n_samples=200, noise=0.1, random_state=42)
# Change labels to {-1, 1} for AdaBoost
y = np.where(y == 0, -1, 1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

ada = AdaBoostClassifier(n_estimators=10)
ada.fit(X_train, y_train)
print(f"Our AdaBoost Accuracy: {ada.score(X_test, y_test):.4f}")

sk_ada = SklearnAda(n_estimators=10, algorithm='SAMME')
sk_ada.fit(X_train, y_train)
print(f"Sklearn AdaBoost Accuracy: {sk_ada.score(X_test, y_test):.4f}")

## 4. Visualizing Decision Boundaries

In [ ]:
h = 0.02
x_min, x_max = X[:, 0].min() - 0.5, X[:, 0].max() + 0.5
y_min, y_max = X[:, 1].min() - 0.5, X[:, 1].max() + 0.5
xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))

Z = ada.predict(np.c_[xx.ravel(), yy.ravel()])
Z = Z.reshape(xx.shape)

plt.contourf(xx, yy, Z, cmap=plt.cm.RdBu, alpha=0.8)
plt.scatter(X[:, 0], X[:, 1], c=y, cmap=plt.cm.RdBu, edgecolors='k')
plt.title("AdaBoost Decision Boundary")
plt.show()